This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with a simple model, gain corruptions and a gain solver,
then again with a more complicated sky and beam model,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "h5parm_name": "demo.h5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-12 13:29:44,158 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-12 13:29:44,159 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2025-02-12 13:29:44,159 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-12 13:29:44,174 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-12 13:29:44,745 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-12 13:29:44,898 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-12 13:29:45,332 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-12 13:29:45,333 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-12 13:29:46,890 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-12 13:29:46,891 - pipeline.bandpass_calibration - INFO - end_to_end_subbands = true
2025-02-12 13:29:47,141 

pipeline ran in 4.9 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "h5parm_name": "demo.h5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-12 13:29:49,065 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-12 13:29:49,066 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-12 13:29:49,067 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-12 13:29:49,068 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-12 13:29:49,069 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-12 13:29:49,069 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-12 13:29:49,070 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-12 13:29:49,663 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-12 13:29:49,664 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-12 13:29:49,686 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-12 13:29:49,738 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-12 13:29:52,763 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-12 13:29:53,325 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-12 13:29:53,326 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-12 13:29:54,145 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-12 13:29:54,147 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-12 13:29:54,148 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-12 13:29:54,148 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-12 13:29:54,812 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-12 13:29:54,813 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-12 13:29:54,813 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-12 13:

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-12 13:29:58,057 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.h5
2025-02-12 13:29:58,058 - workflow.utils - INFO - exporting cal solutions to demo.h5
2025-02-12 13:29:58,070 - workflow.utils - INFO - output dimensions: {'time': 1, 'ant': 20, 'freq': 64, 'pol': 4}
2025-02-12 13:29:58,080 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-12 13:29:58,082 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 9.4 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "h5parm_name": "demo2.h5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-12 13:29:58,484 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-12 13:29:58,485 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-12 13:29:58,486 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-12 13:29:58,488 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-12 13:29:58,489 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-12 13:29:58,489 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-12 13:29:58,490 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-12 13:29:59,101 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-12 13:29:59,102 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-12 13:29:59,111 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-12 13:29:59,143 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-12 13:30:02,479 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-12 13:30:02,480 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2025-02-12 13:30:03,035 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-12 13:30:03,036 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-12 13:30:03,833 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-12 13:30:03,835 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-12 13:30:03,836 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-12 13:30:03,837 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-12 13:30:04,539 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-12 13:30:04,540 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-12 13:30:04

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-12 13:30:08,722 - pipeline.bandpass_calibration - INFO - Writing solutions to demo2.h5
2025-02-12 13:30:08,725 - workflow.utils - INFO - exporting cal solutions to demo2.h5
2025-02-12 13:30:08,734 - workflow.utils - INFO - output dimensions: {'time': 1, 'ant': 20, 'freq': 64, 'pol': 4}
2025-02-12 13:30:08,742 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-12 13:30:08,744 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.7 seconds
